<a href="https://colab.research.google.com/github/schmellerl/gradient_flows_order_parameters_mechanics/blob/main/colab/Example3_Output.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# put in a separate file
try:
    import google.colab  # noqa: F401
except ImportError:
    import ufl
    import dolfin
else:
    try:
        import ufl
        import dolfin
    except ImportError:
        !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
        import ufl
        import dolfin

from google.colab import drive
drive.mount('/content/drive', force_remount=True)
input_path  = '/content/drive/MyDrive/ColabNumerics/Example3/'
output_path = '/content/drive/MyDrive/ColabNumerics/Images/'

In [ ]:
from fenics import *
from matplotlib import pyplot as plt
import numpy as np

plt.rcParams.update({'font.size': 14})

E = np.load(input_path + "energies.npy")
mesh = Mesh(input_path + "mesh.xml")

Vu    = FunctionSpace(mesh,VectorElement("P", mesh.ufl_cell(), 1))
Vpsi  = FunctionSpace(mesh,FiniteElement("P", mesh.ufl_cell(), 1))

m = 0
plt.figure(figsize=(4*7,7))

for i in [10,100,200,1000]:
  mesh = Mesh(input_path + "mesh.xml")

  psi1 = Function(Vpsi,input_path + "psi1"+str(i)+".xml")
  psi2 = Function(Vpsi,input_path + "psi2"+str(i)+".xml")
  u    = Function(Vu  ,input_path + "u"+str(i)+".xml")

  psi3 = project(-1-psi1-psi2,Vpsi)
  psi4 = project( (1+psi1)/2 + 2*(1+psi2)/2 + 3*(1+psi3)/2 +1/20 ,Vpsi)

  ALE.move(mesh,u)
  m += 1

  plt.subplot(1,4,m)
  c = plot(psi4,mode="color",vmin=1,vmax=3.1,cmap='jet')

  plt.xlim([0,5])
  plt.ylim([0,5])
  
  plt.title("time $t="+f"{E[1][i]:.4f}"+"$")
  plt.xlabel("$X$")
  plt.ylabel("$Y$")

cax = plt.axes([0.915, 0.17, 0.015, 0.67]) 
plt.colorbar(c,cax=cax)
plt.savefig(output_path + "EX3_FIG1.jpeg",dpi=800)
plt.show()


plt.figure(figsize=(6,6))
mesh = Mesh(input_path + "mesh.xml")
u    = Function(Vu  ,input_path + "u"+str(i)+".xml")
ALE.move(mesh,u)
c = plot(psi4,mode="color",vmin=1,vmax=3,cmap='jet')
plot(mesh,linewidth=0.3)
plt.ylim([0,6])
plt.savefig(output_path + "EX3_FIG3.pdf")

plt.figure(figsize=(6,6))
plt.plot(E[1],E[2])
plt.grid(True)
plt.xlabel("time $t$")
plt.ylabel("energy")
plt.legend(['$\mathscr{F}$'],fontsize=14)
plt.savefig(output_path + "EX3_FIG2.pdf")